In [19]:
# 필터 주피터입니다.
# 우선 Step 3개로 준비했습니다.
# Step.1 5달러 이하 종목 삭제
# Step.2 20일 평균 거래 금액 5,000만달러 이하 삭제
# Step.3 변동률 순위 구하기

In [1]:
import ccxt
import pandas as pd

In [3]:
# 로그인

binance = ccxt.binance({
    #'apiKey' : 'MPe1dhkEUjkk2FEewGDrIOUK5tMKy0f9QhwWNSRYBWjuMQvX2oMb28H4rxf8K5vA',
    #'secret' : 'p26Mb62MBiNutb6M6RYaHGNZCiVUxe92OogQrPBpVEml0nHKuRshcS19Z0HHfkkI',
    'enableRateLimit' : True,
    # 'options' : {
    #     'defaultType' : 'future'
    # }
})
# markets = binance.load_markets()
markets = binance.fetch_tickers()
markets = pd.DataFrame(markets)
# markets = markets.drop(['BTCSTUSDT', 'BTCUSDT_220930', 'ETHUSDT_220930', 'UNFI/USDT'],axis=1)
# markets = markets.reset_index(drop=True)

In [9]:
# Step.1 5달러 이하 종목 삭제

tickers = []
tickers_5 = []
for i in range(len(markets.columns)):
    if "/USDT" in markets.iloc[0][i]:
        ticker = markets.iloc[0][i]
        tickers.append(ticker)
for i in range(len(tickers)):
    ticker = tickers[i]
    ticker_now_price = binance.fetch_ticker(ticker)
    ticker_price = ticker_now_price['close']
    if ticker_price > 5:
        tickers_5.append(ticker)
# ticker_5에 5달러 이상 종목 저장
print(tickers_5)

['BTC/USDT', 'ETH/USDT', 'BNB/USDT', 'NEO/USDT', 'LTC/USDT', 'ETC/USDT', 'BCH/USDT', 'LINK/USDT', 'XMR/USDT', 'ZEC/USDT', 'DASH/USDT', 'ATOM/USDT', 'FTT/USDT', 'SOL/USDT', 'REP/USDT', 'COMP/USDT', 'ZEN/USDT', 'MKR/USDT', 'DCR/USDT', 'BNBUP/USDT', 'YFI/USDT', 'BAL/USDT', 'NMR/USDT', 'DOT/USDT', 'PAXG/USDT', 'WNXM/USDT', 'TRB/USDT', 'YFII/USDT', 'KSM/USDT', 'EGLD/USDT', 'DOTDOWN/USDT', 'WING/USDT', 'UNI/USDT', 'AVAX/USDT', 'AAVE/USDT', 'FIL/USDT', 'AXS/USDT', 'UNFI/USDT', 'JUV/USDT', 'PSG/USDT', 'OG/USDT', 'BTCST/USDT', 'AUTO/USDT', 'BTG/USDT', 'BAR/USDT', 'ICP/USDT', 'AR/USDT', 'LPT/USDT', 'TORN/USDT', 'BOND/USDT', 'MLN/USDT', 'QNT/USDT', 'FARM/USDT', 'QUICK/USDT', 'GNO/USDT', 'ILV/USDT', 'LAZIO/USDT', 'AUCTION/USDT', 'BNX/USDT', 'MOVR/USDT', 'CITY/USDT', 'ENS/USDT', 'KP3R/USDT', 'PORTO/USDT', 'ALCX/USDT', 'SANTOS/USDT', 'FXS/USDT', 'CVX/USDT', 'ALPINE/USDT', 'BIFI/USDT']


In [11]:
# Step.2 20일 평균 거래 금액 5,000만달러 이하 삭제
j = 0
volume_total = pd.DataFrame(columns=['ticker', 'volume'])
while True:
    # j번째 USDT 종목을 선택한다.
    ticker = tickers_5[j]
    # 티커의 자료를 받는다.
    df = binance.fetch_ohlcv(ticker, timeframe='1d', limit=20)
    df = pd.DataFrame(df, columns=['date', 'open',
                  'high', 'low', 'close', 'volume'])
    # 이전 ticker의 값 초기화
    dolor_value = []
    dolor_total = 0
    # 하루 거래량 구한다.
    for i in range(len(df)):
        dolor = float(df.loc[i, 'close']) * float(df.loc[i, 'volume'])
        dolor_value.append(dolor)
    df.insert(len(df.columns), "dolor", dolor_value)
    # 20일 평균 거래량 구한다.
    for i in range(len(df)):
        dolor_total += df.loc[i, 'dolor']
    dolor_total = int(dolor_total)
    # 3억달러로 변경
    if dolor_total > 300000000:
        volume_total.loc[j] = [ticker, dolor_total]
    j += 1
    if j == len(tickers_5):
        break

# 인덱스 리셋 (key값 오류뜸)
volume_total = volume_total.reset_index(drop=True)
print(volume_total)


         ticker       volume
0      BTC/USDT  86237470522
1      ETH/USDT  27065612170
2      BNB/USDT   2592418129
3      LTC/USDT    651168409
4      ETC/USDT   2493242707
5     LINK/USDT    577756188
6     ATOM/USDT    948350070
7      SOL/USDT   2310718431
8      DOT/USDT    719780275
9      UNI/USDT    306518271
10    AVAX/USDT   1141202429
11     FIL/USDT    674685690
12    UNFI/USDT    444589861
13   BTCST/USDT    307452402
14   LAZIO/USDT    422092698
15   PORTO/USDT    488815390
16  SANTOS/USDT    759448257


In [12]:
# Step.3 변동률 순위 구하기
# cs를 저장할 데이터 프레임
j = 0
USDT_cs = pd.DataFrame(columns=['ticker', 'cs'])
while True:
    # j번째 USDT 종목을 선택한다.
    ticker = volume_total.loc[j, 'ticker']
    # 티커의 자료를 받는다.
    df = binance.fetch_ohlcv(ticker, timeframe='1d', limit=200)
    # cs와 dpc 초기화
    dpc_total = []
    cs_total = []
    df = pd.DataFrame(df, columns=['date', 'open',
                  'high', 'low', 'close', 'volume'])
    for i in range(len(df)):
        # dpc를 추가하는 코드
        if i > 0:
            dpc = (float(df.loc[i, 'close']) - float(df.loc[i -
               1, 'close']))/float(df.loc[i-1, 'close'])*100
            dpc_total.append(dpc)
        else:
            dpc = 0
            dpc_total.append(dpc)
    df.insert(len(df.columns), "dpc", dpc_total)
    
    for i in range(len(df)):
        # cs를 추가하는 코드
        if i > 0:
            cs = float(df.loc[i, 'dpc']) + float(cs)
            cs_total.append(cs)
        else:
            cs = 0
            cs_total.append(cs)
    df.insert(len(df.columns), 'cs', cs_total)
    
    # cs값을 절대값으로 구하는 코드
    cs_value = abs(float(df.loc[len(df)-1, 'cs']))
    print('{}의 200일간 변동률 = {}'.format(ticker, cs_value))
    USDT_cs.loc[j] = [ticker, cs_value]
    j += 1
    if j == len(volume_total):
        break
USDT_cs_sort = USDT_cs.sort_values('cs', ascending=False)
USDT_cs_sort = USDT_cs_sort.reset_index(drop=True)
USDT_cs_sort

BTC/USDT의 200일간 변동률 = 66.1034607187482
ETH/USDT의 200일간 변동률 = 46.565676361378
BNB/USDT의 200일간 변동률 = 27.893777312893235
LTC/USDT의 200일간 변동률 = 51.76170995074614
ETC/USDT의 200일간 변동률 = 45.3947505359809
LINK/USDT의 200일간 변동률 = 61.582491540992905
ATOM/USDT의 200일간 변동률 = 47.80776770977095
SOL/USDT의 200일간 변동률 = 82.23630330705741
DOT/USDT의 200일간 변동률 = 70.07354515088541
UNI/USDT의 200일간 변동률 = 20.402267770903958
AVAX/USDT의 200일간 변동률 = 120.55117980625806
FIL/USDT의 200일간 변동률 = 86.26177373038853
UNFI/USDT의 200일간 변동률 = 449.8016223907868
BTCST/USDT의 200일간 변동률 = 68.32042360635522
LAZIO/USDT의 200일간 변동률 = 147.34810685709604
PORTO/USDT의 200일간 변동률 = 137.23154859017382
SANTOS/USDT의 200일간 변동률 = 313.2686723749046


,ticker,cs
0,UNFI/USDT,449.801622
1,SANTOS/USDT,313.268672
2,LAZIO/USDT,147.348107
3,PORTO/USDT,137.231549
4,AVAX/USDT,120.551180
5,FIL/USDT,86.261774
6,SOL/USDT,82.236303
7,DOT/USDT,70.073545
8,BTCST/USDT,68.320424
9,BTC/USDT,66.103461


In [25]:
sort_list = []
for sort in range(10):
    sort_list.append(USDT_cs_sort.iloc[sort][0])
sort_list[0]

'HNT/USDT'

In [26]:
USDT_cs_sort.iloc[0][1]

108.09451751305818